In [7]:
!pip install catboost
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style='darkgrid', font_scale=2)
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder


from sklearn.model_selection import cross_val_score, learning_curve

import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
# Reason behind using CatBoost is explained later in the notebook


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
train_df=pd.read_csv("/content/drive/MyDrive/SpaceShipFolder/train.csv")
train_df

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
test_df=pd.read_csv("/content/drive/MyDrive/SpaceShipFolder/test.csv")
test_df

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,Earth,True,G/1496/S,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0,Jeron Peter
4273,9269_01,Earth,False,NaN,TRAPPIST-1e,42.0,False,0.0,847.0,17.0,10.0,144.0,Matty Scheron
4274,9271_01,Mars,True,D/296/P,55 Cancri e,NaN,False,0.0,0.0,0.0,0.0,0.0,Jayrin Pore
4275,9273_01,Europa,False,D/297/P,NaN,NaN,False,0.0,2680.0,0.0,0.0,523.0,Kitakan Conale


In [11]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [12]:
train_df[["HomePlanet","Transported"]].groupby(["HomePlanet"]).mean().sort_values(by='Transported',ascending=False)

,Transported
HomePlanet,
Europa,0.658846
Mars,0.523024
Earth,0.423946


In [13]:
train_df[["CryoSleep","Transported"]].groupby(["CryoSleep"]).mean().sort_values(by='Transported',ascending=False)

,Transported
CryoSleep,
True,0.817583
False,0.328921


In [14]:
train_df[["Cabin","Transported"]].groupby(["Cabin"]).mean().sort_values(by='Transported',ascending=False)

,Transported
Cabin,
T/3/P,1.0
E/513/S,1.0
G/1012/P,1.0
G/1011/S,1.0
E/516/P,1.0
...,...
F/797/P,0.0
F/797/S,0.0
F/1077/P,0.0


In [15]:
train_df[["VIP","Transported"]].groupby(["VIP"]).mean().sort_values(by='Transported',ascending=False)

,Transported
VIP,
False,0.506332
True,0.381910


In [16]:
train_df.set_index('PassengerId',inplace=True)
test_df.set_index('PassengerId',inplace=True)

In [17]:
train_df[['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']] =train_df[['RoomService','FoodCourt',
                                                                         'ShoppingMall','Spa','VRDeck']].fillna(0)
test_df[['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']] =test_df[['RoomService','FoodCourt',
                                                                       'ShoppingMall','Spa','VRDeck']].fillna(0)
train_df['Age'] = train_df['Age'].fillna(train_df['Age'].mean())
test_df['Age'] = test_df['Age'].fillna(test_df['Age'].mean())

In [18]:
train_df['VIP'] =train_df['VIP'].fillna(False)
test_df['VIP'] =test_df['VIP'].fillna(False)

In [19]:
train_df['HomePlanet'] =train_df['HomePlanet'].fillna('Europa')
test_df['HomePlanet'] =test_df['HomePlanet'].fillna('Europa')

In [20]:
train_df['CryoSleep'].value_counts()

False    5439
True     3037
Name: CryoSleep, dtype: int64

In [21]:
train_df['Destination'].value_counts()

TRAPPIST-1e      5915
55 Cancri e      1800
PSO J318.5-22     796
Name: Destination, dtype: int64

In [22]:
train_df['Destination']=train_df['Destination'].fillna("PSO J318.5-22")
test_df['Destination']=test_df['Destination'].fillna("PSO J318.5-22")

train_df['CryoSleep'] =train_df['CryoSleep'].fillna(False)
test_df['CryoSleep'] =test_df['CryoSleep'].fillna(False)

train_df['Cabin'] =train_df['Cabin'].fillna('T/0/P')
test_df['Cabin'] =test_df['Cabin'].fillna('T/0/P')

In [23]:
train_df.isnull().sum().sort_values(ascending=False)


Name            200
HomePlanet        0
CryoSleep         0
Cabin             0
Destination       0
Age               0
VIP               0
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Transported       0
dtype: int64

In [24]:
train_df.corr()['Transported'].sort_values(ascending=False)[1:]


CryoSleep       0.460132
FoodCourt       0.045583
ShoppingMall    0.009391
VIP            -0.037261
Age            -0.074249
VRDeck         -0.204874
Spa            -0.218545
RoomService    -0.241124
Name: Transported, dtype: float64

In [25]:
train_df[['Deck','Num','Side']] = train_df.Cabin.str.split('/',expand=True) 
test_df[['Deck','Num','Side']] = test_df.Cabin.str.split('/',expand=True)

In [26]:
train_df['Total Spent'] =train_df['RoomService']+train_df['FoodCourt']+ train_df['ShoppingMall']+ train_df['Spa']+ train_df['VRDeck']
test_df['Total Spent'] = test_df['RoomService']+test_df['FoodCourt']+ test_df['ShoppingMall']+ test_df['Spa']+ test_df['VRDeck']

In [27]:
train_df['AgeGroup']  = 0
for i in range(6):
    train_df.loc[(train_df.Age >= 10*i) & (train_df.Age < 10*(i+1)),'AgeGroup'] = i
    
test_df['AgeGroup']  = 0
for i in range(6):
    test_df.loc[(test_df.Age >= 10*i) & (test_df.Age < 10*(i+1)),'AgeGroup'] = i
train_df

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Deck,Num,Side,Total Spent,AgeGroup
PassengerId,,,,,,,,,,,,,,,,,,
0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,B,0,P,0.0,3
0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,F,0,S,736.0,2
0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,A,0,S,10383.0,5
0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,A,0,S,5176.0,3
0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,F,1,S,1091.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False,A,98,P,8536.0,4
9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False,G,1499,S,0.0,1
9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True,G,1500,S,1873.0,2


In [28]:
from sklearn.preprocessing import LabelEncoder
categorical_cols= ['HomePlanet','CryoSleep','Destination','VIP','Deck','Side','Num']
for i in categorical_cols:
    le=LabelEncoder()
    arr=np.concatenate((train_df[i], test_df[i])).astype(str)
    le.fit(arr)
    train_df[i]=le.transform(train_df[i].astype(str))
    test_df[i]=le.transform(test_df[i].astype(str))



In [29]:
train_df=train_df.drop(['Name','Cabin'],axis=1)
test_df= test_df.drop(['Name','Cabin'],axis=1)

In [30]:
X= train_df.drop('Transported',axis = 1)
y = train_df['Transported']

In [31]:
train_df['Transported']=train_df['Transported'].replace({True:1,False:0})
train_df

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Deck,Num,Side,Total Spent,AgeGroup
PassengerId,,,,,,,,,,,,,,,,
0001_01,1,0,2,39.0,0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0.0,3
0002_01,0,0,2,24.0,0,109.0,9.0,25.0,549.0,44.0,1,5,0,1,736.0,2
0003_01,1,0,2,58.0,1,43.0,3576.0,0.0,6715.0,49.0,0,0,0,1,10383.0,5
0003_02,1,0,2,33.0,0,0.0,1283.0,371.0,3329.0,193.0,0,0,0,1,5176.0,3
0004_01,0,0,2,16.0,0,303.0,70.0,151.0,565.0,2.0,1,5,1,1,1091.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9276_01,1,0,0,41.0,1,0.0,6819.0,0.0,1643.0,74.0,0,0,1872,0,8536.0,4
9278_01,0,1,1,18.0,0,0.0,0.0,0.0,0.0,0.0,0,6,556,1,0.0,1
9279_01,0,0,2,26.0,0,0.0,0.0,1872.0,1.0,0.0,1,6,559,1,1873.0,2


In [32]:
model = GradientBoostingClassifier()

model.fit(X,y)  


GradientBoostingClassifier()

In [33]:
prediction = model.predict(test_df)
sub = pd.DataFrame({'Transported':prediction.astype(bool)}, index = test_df.index)

sub.head()
sub.to_csv('submission11.csv')


